In [1]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [0]:
data = pd.read_csv("indo_dataset.csv")

In [5]:
data.iloc[0]

index                                                       0
sentimen                                                    0
Tweet       lagu bosan apa yang aku save ni huhuhuhuhuhuhu...
Name: 0, dtype: object

In [0]:
max_fatures = 2500
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['Tweet'].values)

In [0]:
X = tokenizer.texts_to_sequences(data['Tweet'].values)

In [8]:
X[0]

[629, 983, 15, 3, 1, 1735, 17]

In [0]:
X = pad_sequences(X)

In [0]:
Y = pd.get_dummies(data['sentimen']).values

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)

In [12]:
embed_dim = 128

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(LSTM(256, dropout=0.5, recurrent_dropout=0.5))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 25, 128)           320000    
_________________________________________________________________
lstm (LSTM)                  (None, 256)               394240    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                16448     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 130       
Total params: 730,818
Trainable params: 730,818
Non-trainable params: 0
__________________________________________________

In [13]:
batch_size = 32
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs = 10, batch_size=batch_size)

Train on 3670 samples, validate on 1809 samples
Epoch 1/10
3670/3670 [==============================] - 26s 7ms/sample - loss: 0.6886 - accuracy: 0.5368 - val_loss: 0.6647 - val_accuracy: 0.6130
Epoch 2/10
3670/3670 [==============================] - 23s 6ms/sample - loss: 0.5991 - accuracy: 0.6831 - val_loss: 0.5867 - val_accuracy: 0.6932
Epoch 3/10
3670/3670 [==============================] - 23s 6ms/sample - loss: 0.4378 - accuracy: 0.7975 - val_loss: 0.6114 - val_accuracy: 0.6866
Epoch 4/10
3670/3670 [==============================] - 22s 6ms/sample - loss: 0.3684 - accuracy: 0.8455 - val_loss: 0.7151 - val_accuracy: 0.7324
Epoch 5/10
3670/3670 [==============================] - 22s 6ms/sample - loss: 0.3056 - accuracy: 0.8782 - val_loss: 0.6835 - val_accuracy: 0.7164
Epoch 6/10
3670/3670 [==============================] - 23s 6ms/sample - loss: 0.2473 - accuracy: 0.9019 - val_loss: 0.8741 - val_accuracy: 0.7253
Epoch 7/10
3670/3670 [==============================] - 22s 6ms/sample

In [0]:
X_T = tokenizer.texts_to_sequences(np.array(["saya benci baso", "saya suka baso"]))
X_T = pad_sequences(X_T, maxlen=25)

In [15]:
for d in np.argmax(model.predict(X_T), axis=0):
  print(["Negativ", "Positif"][d])

Negativ
Positif


In [0]:
model.save_weights('lstm.h5')

In [17]:
model2 = Sequential()
model2.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model2.add(LSTM(256))
model2.add(Dense(64, activation='relu'))
model2.add(Dense(2, activation='softmax'))
model2.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model2.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 128)           320000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               394240    
_________________________________________________________________
dense_2 (Dense)              (None, 64)                16448     
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 130       
Total params: 730,818
Trainable params: 730,818
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
model2.load_weights('lstm.h5')

In [19]:
for d in np.argmax(model2.predict(X_T), axis=0):
  print(["Negativ", "Positif"][d])

Negativ
Positif
